In [4]:
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import WebDriverException

In [5]:
def getPlaylist(playlist_tag, num_max):

    pageIndex = 1 # page 번호 초기화
    tmp_Playlist = []  # 플레이리스트 리스트 초기화

    while len(tmp_Playlist) < num_max:
        # 웹페이지 로드 - url에 tag와 page 넣어서 이동
        driver.get(f'https://www.melon.com/dj/djfinder/djfinder_inform.htm?djSearchType=T&djSearchKeyword={playlist_tag}#params[djSearchType]=T&params[djSearchKeyword]=%23{playlist_tag}&params[orderBy]=POP&params[pagingFlag]=Y&params[tagSearchType]=S&po=pageObj&startIndex={pageIndex}')
        time.sleep(5)

        playlist_entries = driver.find_elements(By.XPATH, '//*[@id="djPlylstList"]/div/ul/li')
        print(playlist_entries[1])
        if not playlist_entries:
            # 플레이리스트가 없는 경우 종료
            break

        for pl_entry in playlist_entries:
            print(pl_entry)
            # plylstSeq : 플레이리스트 고유번호
            # plylstSeq를 이용해 url 저장
            plylstSeq = pl_entry.find_element(By.XPATH, './div[2]/div[2]/button').get_attribute('data-djcol-no')
            pl_href = f'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq={plylstSeq}'
            
            try:
                pl_title = pl_entry.find_element(By.XPATH, './div[2]/div[1]/a[2]').text
                pl_title = pl_title.apply(lambda x: re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\s!@#$%^&*_+=[\]{}|]', '', str(x)))
            except Exception as e:
                pl_title = ''
                print(str(e))

            # pl_len : 플레이리스트 길이
            pl_len = pl_entry.find_element(By.XPATH, './div[2]/div[2]/span').text.replace('곡','')
            # pl_len : 플레이리스트 태그
            pl_tag = pl_entry.find_element(By.XPATH, './div[2]/div[3]/a/span').text
            
            # 각각의 플레이리스트마다 정보 - [플리id, 플리이름, 플리링크, 플리길이, 플리정보]
            info_Playlist = [plylstSeq, pl_title, pl_href, pl_len, pl_tag]

            # 양상마다 tmp_Playlist 를 전체 All_Playlist에 저장
            tmp_Playlist.append(info_Playlist)

        pageIndex += 20

    return tmp_Playlist

In [ ]:
## 플리 추천
playlist_tag = ['우울', '슬픔', '위로', '외로움', '힐링', '편안', '잔잔', '산책', 
                            '무기력', '지칠때', '충전', '힐링', '평화', '피로', '잠', '불면',
                            '짜증', '집중', '자존감', '울고싶은날', '절망', '죽고싶을때', '불안']

playlist_dict = {
                                 '우울': playlist_tag[0],  '슬픔': playlist_tag[1, 2, 19],  '외로움': playlist_tag[3, 4],  '분노': playlist_tag[16],  '무기력': playlist_tag[8, 9, 10],  '의욕상실': playlist_tag[8, 9, 10],
                                 '식욕저하': playlist_tag[8, 9, 10],  '식욕증가': [8, 9, 10],  '감정조절이상': playlist_tag[12],  '불면': playlist_tag[13, 14],  '초조': playlist_tag[5, 6, 7],  '피로': playlist_tag[13, 14, 15],
                                 '죄책감': playlist_tag[2],  '집중력저하': playlist_tag[17],  '자신감저하': playlist_tag[18],  '자존감저하': playlist_tag[18],  '절망': playlist_tag[18, 2],  '자살': playlist_tag[20],  '불안': playlist_tag[22, 5]
                            }

In [ ]:
num_max = 200

CHROMEDRIVER_PATH = './chromedriver.exe' # ChromeDriver 설치 위치
service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service)

# 빈 리스트 생성
All_Playlist = []  # 전체 플레이리스트 리스트

for t in playlist_tag:
    tmp_list = getPlaylist(t, num_max)
    All_Playlist.append(tmp_list)

driver.quit()

In [ ]:
#  csv 저장 
for i in range(len(All_Playlist)):
    df = pd.DataFrame(All_Playlist, columns=['id', 'playlist_name', 'href', 'num of songs', 'hashtag'])
    df.to_csv(f'{file_name}.csv', encoding='utf-8', index=False)

In [ ]:
# # 저장 후 nan 확인해보기
import pandas as pd

for file_name in playlist_tag:
    # 데이터프레임 불러오기
    df = pd.read_csv(f'{file_name}.csv')
    nan_check = df.isnull().sum()
    if nan_check.any() == True:
        print("NaN")